In [ ]:
%pip install pandas langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt langchain_upstage

In [ ]:
import os
import pandas as pd
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

GPT_API_KEY = os.environ["GPT_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
UPSTAGE_API_KEY = os.environ["UPSTAGE_API_KEY"]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [ ]:
folder_path = '../data'

In [ ]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

In [ ]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [ ]:
from langchain_pinecone import PineconeVectorStore

index_name = 'upstage-index'

In [ ]:
# DB 처음 만들 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


In [ ]:
# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [ ]:
query = '결제 후 주문 취소가 가능한가요?'

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

# LLM 질의 테스트

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

    

In [ ]:
# 유사도 보기
# results = database.similarity_search_with_score(query=query, k=3)
# pprint(results)
query = '결제 시 카드 정보는 어떻게 보호되나요?'
results = database.similarity_search_with_score(query=query, k=3)
pprint(results)

print()
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:  
    print(ans + '.')

# OLLAMA TEST

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.
또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.

쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOllama(
    model="llama3.1:8b"
)

def merge_pages(pages):
    merged = "".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)
# 유사도 보기
# results = database.similarity_search_with_score(query=query, k=3)
# pprint(results)
query = '결제 후 주문 취소가 가능한가요?'
results = database.similarity_search_with_score(query=query, k=3)
pprint(results)

print()
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:  
    print(ans + '.')
    